# EMS Prediction

**Use this notebook to score YOUR genetic variants for functional impact.** After understanding the methodology from the [EMS Training Tutorial](https://statfungen.github.io/xqtl-protocol/code/xqtl_modifier_score/ems_training.html), apply the trained model to your own variant lists.

## What This Does
- **Input**: Your variant list (format: `chr:pos:ref:alt`)
- **Output**: Same variants with EMS functional scores (0-1 probability)
- **Purpose**: Prioritize variants likely to affect gene expression

## How to Run the EMS Pipeline

**Step 1: Prepare your variant file**
```
variant_id
2:12345:A:T
2:67890:G:C
2:11111:T:A
```

**Step 2: Run the training/prediction pipeline**
```bash
cd ~/xqtl-protocol/code/xqtl_modifier_score/
python model_training_model5_only.py Mic_mega_eQTL 2 \
  --data_config data_config.yaml \
  --model_config model_config.yaml
```

**Step 3: Get your results**
- Trained model: `model_standard_subset_weighted_chr_chr2_NPR_10.joblib`
- Predictions: `predictions_weighted_model_chr2.tsv`
- Performance: `summary_dict_catboost_weighted_model_chr_chr2_NPR_10.pkl`

## What the Model Learned (from Training Documentation)

**Algorithm**: Feature-weighted CatBoost classifier
- **Performance**: 89.78% AUC, 50.5% Average Precision
- **Training data**: 3,056 variants, 4,839 genomic features
- **Top predictor**: Distance to transcription start site (23.58 importance)
- **Key features**: Cell-type regulatory signals, population genetics

**Feature weighting strategy**: Experimental features weighted 10x higher than computational predictions

## Inspect Your Results

**After running the pipeline, you'll have these outputs:**

In [ ]:
import pandas as pd
import joblib

# Load your trained model (after running the pipeline)
MODEL_PATH = "../../data/Mic_mega_eQTL/model_results/model_standard_subset_weighted_chr_chr2_NPR_10.joblib"
trained_model = joblib.load(MODEL_PATH)

print(f"✅ Your trained model: {trained_model.__class__.__name__}")
print(f"Features: {trained_model.feature_count_}")
print(f"Classes: {list(trained_model.classes_)}")

In [ ]:
# Load your prediction results (after running the pipeline)
RESULTS_PATH = "../../data/Mic_mega_eQTL/model_results/predictions_parquet_catboost/predictions_weighted_model_chr2.tsv"
results = pd.read_csv(RESULTS_PATH, sep='\t')

print(f"📊 Results for {len(results)} variants")
print("\nColumns available:")
for col in results.columns:
    if 'pred' in col.lower() or 'score' in col.lower():
        print(f"   - {col}")

# Show example results
print("\nExample predictions:")
display_cols = ['variant_id', 'standard_subset_weighted_pred_prob', 'standard_subset_weighted_pred_label']
print(results[display_cols].head())

## Interpret Your EMS Scores

**Score meaning**: `standard_subset_weighted_pred_prob` = Probability variant affects gene expression

### Priority Guidelines:
- **High Priority (>0.8)**: Strong functional evidence, prioritize for experiments
- **Medium Priority (0.5-0.8)**: Moderate evidence, worth investigating  
- **Low Priority (<0.5)**: Limited functional evidence

### Model Basis (from Training):
- **Distance to gene start**: Primary predictor (closest = most likely functional)
- **Cell-type signals**: Microglia-specific regulatory activity
- **Population genetics**: Allele frequency and constraint data

In [ ]:
# Analyze your results
score_col = 'standard_subset_weighted_pred_prob'

print("🎯 YOUR RESULTS SUMMARY")
print("=" * 30)
print(f"Total variants: {len(results)}")
print(f"High priority (>0.8): {len(results[results[score_col] > 0.8])}")
print(f"Medium priority (0.5-0.8): {len(results[(results[score_col] > 0.5) & (results[score_col] <= 0.8)])}")
print(f"Low priority (<0.5): {len(results[results[score_col] <= 0.5])}")

# Show top-scoring variants
top_variants = results.nlargest(5, score_col)
print("\n🏆 Top 5 variants:")
for _, row in top_variants.iterrows():
    print(f"   {row['variant_id']}: {row[score_col]:.4f}")

## Using Results for Research

### Immediate Actions:
1. **Filter high-priority variants** (score > 0.8) for experimental validation
2. **Cross-reference with literature** for known functional variants
3. **Design follow-up experiments** (luciferase assays, CRISPR validation)

### Export Prioritized Lists:

In [ ]:
# Export high-priority variants for follow-up
high_priority = results[results[score_col] > 0.8]

if len(high_priority) > 0:
    output_file = "high_priority_variants_for_validation.tsv"
    high_priority.to_csv(output_file, sep='\t', index=False)
    print(f"📁 Saved {len(high_priority)} high-priority variants to: {output_file}")
    print("   Ready for experimental design and validation")
else:
    print("ℹ️  No variants with score > 0.8 found")
    print("   Consider lowering threshold or reviewing medium-priority variants")

## For New Variant Lists

**To score additional variants:**

1. **Prepare new variant file** in same format (`chr:pos:ref:alt`)
2. **Update data paths** in `data_config.yaml` to point to your new file
3. **Re-run the pipeline** with same command
4. **Get new predictions** in same output format

**The trained model will automatically:**
- Parse variant coordinates
- Generate genomic features
- Apply feature weighting
- Score functional probability
- Output results for analysis

### Model Performance Expectations:
- **Chromosome 2 variants**: Optimal performance (training data)
- **Other chromosomes**: Good generalization expected
- **Different populations**: May need recalibration
- **Non-brain tissues**: Consider cell-type specificity